In [ ]:
import geokit as gk
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# GEOMETRIES
Within the geometries module, different kind of geometry objects can be defined. Those are: 

1. Point
1. MultiPoint
1. LineString
1. MultiLineString
1. Polygon
1. MultiPolygon

Each object consists of its data (most likely the location) and its spatial reference system SRS from the lection before

## Point
Ponts can be defined with the location information (lat, lon) and the SRS information. The SRS is an SRS-object as created in the section before

In [ ]:
aachen_pt = gk.geom.point((6.083, 52.775), srs=gk.srs.EPSG4326)
print(type(aachen_pt))

The Object attributes can be obtaine via *object.Get...*

In [ ]:
print("X value is: ", aachen_pt.GetX())
print("Y value is: ", aachen_pt.GetY())
print("srs is: ", aachen_pt.GetSpatialReference())
print("type is: ", aachen_pt.GetGeometryName())

In [ ]:
# Get the buffer around this point
aachen_buffered_area = aachen_pt.Buffer(1)

print(type(aachen_buffered_area))
print("type is: ", aachen_buffered_area.GetGeometryName())

## Line

A line is defined as a polyline and a SRS. A polyline is defined by a number of points with linear connection.

In [ ]:
# Make a line
aachen_centered_srs = gk.srs.centeredLAEA(6.083, 50.775)

aachen_line = gk.geom.line(
    [
        (5.6086412456393614e-11, 1.0605099246605318e-09),
        (5.5966859847770105e-11, 11124.48031893997),
        (5.584717712977507e-11, 22249.12252108337),
        (71432.80781956247, -66262.41755972906),
        (105796.52244980012, 1069.9939140476542),
    ],
    srs=aachen_centered_srs,
)

## Draw functions
Geom objects can be drawn within geokit with gk.drawGeoms()

In [ ]:
gk.drawGeoms

### Plot geoms
Plot the Bufferd area, the defined point and the defined line within the latlon coordinate system. The point is in Aachen, which is at around (6,52)

In [ ]:
# Draw some geometries
axh = gk.drawGeoms(aachen_buffered_area, figsize=(6, 6))  # , srs=aachen_centered_srs )
axh = gk.drawGeoms(aachen_pt, ax=axh)  # , srs=aachen_centered_srs )
axh = gk.drawGeoms(aachen_line, ax=axh)  # , srs=aachen_centered_srs)
plt.show()

### Plot in different SRS
Same plot as above. This time, the SRS is the Aachen centered with equal area projection. 

In [ ]:
axh = gk.drawGeoms(aachen_buffered_area, figsize=(6, 6), srs=aachen_centered_srs)
axh = gk.drawGeoms(aachen_pt, ax=axh, srs=aachen_centered_srs)
axh = gk.drawGeoms(aachen_line, ax=axh, srs=aachen_centered_srs)
plt.show()

## Get Centroid

Calculates the centroid point of each geom object by obj.Centroid().
The result does NOT contain a srs.

In [ ]:
ctr_polygon = aachen_buffered_area.Centroid()
print("polygon center - X", ctr_polygon.GetX())
print("polygon center - Y", ctr_polygon.GetY())
print("polygon srs", ctr_polygon.GetSpatialReference())

ctr_line = aachen_line.Centroid()
print("line center - X", ctr_line.GetX())
print("line center - Y", ctr_line.GetY())
print("line srs", ctr_line.GetSpatialReference())

The srs can be assigend by obj.AssignSpatialReference

In [ ]:
## GOTCHYA!
# Apply the old srs
ctr_line.AssignSpatialReference(aachen_line.GetSpatialReference())
print("line srs", ctr_line.GetSpatialReference())

## Boundary
Get the boundary of an object by: obj.Boundary()

In [ ]:
# Get a boundary
boundary_polygon = aachen_buffered_area.Boundary()
print(type(boundary_polygon))
print(boundary_polygon.GetGeometryName())

In [ ]:
# draw the boundary with a buffer
axh = gk.drawGeoms(
    boundary_polygon.Buffer(0.2), figsize=(6, 6), srs=aachen_centered_srs
)
gk.drawGeoms(boundary_polygon, figsize=(6, 6), srs=aachen_centered_srs, ax=axh)
plt.show()

## Spatial Relation

1. Contains
1. Crosses
1. Touches
1. Overlaps

[More information here](https://en.wikipedia.org/wiki/Spatial_relation#/media/File:TopologicSpatialRelarions2.png)


In [ ]:
# test for containment
print(aachen_buffered_area.Contains(aachen_pt))
print(aachen_buffered_area.Contains(aachen_line))
print(boundary_polygon.Buffer(0.2).Contains(aachen_buffered_area))
print(boundary_polygon.Buffer(0.2).Crosses(aachen_buffered_area))
print(boundary_polygon.Buffer(0.2).Touches(aachen_buffered_area))
print(boundary_polygon.Buffer(0.2).Overlaps(aachen_buffered_area))

## Transform SRS and get points

Get points within a geometry within a choosen SRS:

1. Transform object to another SRS by: gk.geom.transform( [obj1, obj2], toSRS)
1. Extract gk.geom.extractVerticies

In [ ]:
# Get Points
boundary_polygon_3857 = gk.geom.transform(boundary_polygon, toSRS=gk.srs.EPSG3857)
gk.geom.extractVerticies(boundary_polygon_3857)[:5, :]

In [ ]:
boundary_polygon_3857.SimplifyPreserveTopology(10)

## Intersect and Union

Example for Intersection and Union of two geometries area1 and area2. 
* Intersection of geoms: 'area1.Intersection(area2)'
* Union of geoms: 'area1.Union(area2)'

Important: All geometry operations need to happen in the same SRS!
If they are not in the same srs, change to one srs: area2 = gk.geom.transform(area2, toSRS=area1.GetSpatialReference())

In [ ]:
# Geom 1:
axh = gk.drawGeoms(aachen_buffered_area, figsize=(5, 5), srs=4326)
# Geom 2:
box = gk.geom.box(6, 52, 7, 53, srs=gk.srs.EPSG4326)
gk.drawGeoms(box, figsize=(5, 5), srs=4326, ax=axh)

In [ ]:
# Intersection of geoms
intersect_geom = aachen_buffered_area.Intersection(box)
axh = gk.drawGeoms(intersect_geom, figsize=(5, 5), srs=4326)

In [ ]:
# Union of geoms
unioned_geom = aachen_buffered_area.Union(box)
axh = gk.drawGeoms(unioned_geom, figsize=(5, 5), srs=4326)

## Getting areas

In [ ]:
box = gk.geom.box(6, 52, 7, 53, srs=gk.srs.EPSG4326)
print("Box area in degrees^2:", box.Area())

In [ ]:
ctr = box.Centroid()
box_centered_laea_srs = gk.srs.centeredLAEA(ctr.GetX(), ctr.GetY())
box.TransformTo(box_centered_laea_srs)
print("Box area in meters^2:", box.Area())